In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import numpy.linalg as LA

Setting the Hyper-Parameter:

In [2]:
max_iteration = 10000
weight_decay = 1e-5
NUM_EMPLOYEES = 20500
NUM_TRAIN = 20000
NUM_VAL = 500
Epsilon_1 = 10
Epsilon_2 = 10 
NUM_FEATURES = 3
BANDWIDTH = 0.5

In [3]:
def Generating_Kernel(Feature_Matrix, Kernel_type, power=10, BW=10):
    if Kernel_type == "Gaussian_Kernel":
        square = np.sum(Feature_Matrix ** 2, axis=1)
        column_vec = square[:, np.newaxis]
        row_vec = square[np.newaxis, :]
        Gaussian_Kernel = np.exp(-1 * (-2 * Feature_Matrix.dot(Feature_Matrix.T) + column_vec + row_vec) / (2 * BW ** 2))
        return Gaussian_Kernel
    elif Kernel_type == "Linear_Kernel":
        return X.dot(X.T)
    elif Kernel_type == "Polynomial_Kernel":
        return (X.dot(X.T) + 1) ** power

In [ ]:
X = np.zeros((NUM_EMPLOYEES, NUM_FEATURES))# the training set
for i in range(NUM_EMPLOYEES):
    while (True):
        X[i] =np.random.multivariate_normal(np.zeros(NUM_FEATURES), np.eye(NUM_FEATURES))
        _lambda = Epsilon_1 * np.sum(X[i,:] ** 2) + Epsilon_2 * np.sum(X[i,:])
        if _lambda > 0:
            break

Mi = np.max(X, axis = 0)
mi = np.min(X, axis = 0)
print(Mi, mi)
# X = (X - mi) / (Mi - mi)# rescale to [0, 1]

survival_times = np.zeros(NUM_EMPLOYEES) # the surivial time of each employees
for i in range(NUM_EMPLOYEES):
    age = np.random.exponential(Epsilon_1 * np.sum(X[i,:] ** 2) + Epsilon_2 * np.sum(X[i,:]), size=1)
    survival_times[i] = np.ceil(age)

NUM_TASKS = int(max(survival_times))   

Y = np.ones((NUM_EMPLOYEES, NUM_TASKS)) # the lifetime matrix of all employees, if one employee leave at the time interval k, then from Y[i,k](inlcude)  all entries are -1
for i in range(NUM_EMPLOYEES):
    Y[i, int(survival_times[i])-1:] = -1

Kernel_Matrix = Generating_Kernel(X, "Gaussian_Kernel", BW = BANDWIDTH)
Kernel_Matrix_Train = Kernel_Matrix[:NUM_TRAIN,:NUM_TRAIN]
Kernel_Matrix_Val = Kernel_Matrix[NUM_TRAIN:,:NUM_TRAIN]

[4.47657968 4.3703066  4.29534454] [-4.16416797 -4.20865582 -4.14432688]


In [ ]:
# plt.scatter(range(NUM_FEATURES), X[:,0], X[1], X[2])
# plt.scatter(range(NUM_TRAIN), X[:NUM_TRAIN,0])
# plt.scatter(range(NUM_TRAIN), X[:NUM_TRAIN,1])4
for i in range(100):
    plt.scatter(range(NUM_FEATURES), X[i])


In [ ]:
largerthan035 = np.sum(X >= 0.35)
largerthan064 = np.sum(X >= 0.64)
print((largerthan035 - largerthan064) / (X.shape[0] * X.shape[1])) 

In [ ]:
similarity_vec = Kernel_Matrix_Train[0]
mean_list = []
median_list = []
age_gap_list = []
print("the age of 1st sample is:", survival_times[0])
for i in range(10):
    indices = np.where((similarity_vec >= i*0.1) & (similarity_vec <= (i+1)*0.1)) [0]
    age_gap = abs(survival_times[indices] - survival_times[0])
    num_samples = len(indices)
    print( "There are ", num_samples , "samples of similarity between {:.1f} and {:.1f}".format(0.1 * i, 0.1 * (i+1)),
          "which have an average age_gap ",np.mean(age_gap), 
          "and the median is:",np.median(age_gap))
    mean_list.append(np.mean(age_gap))
    median_list.append(np.median(age_gap))
    age_gap_list.append(age_gap)
plt.boxplot(age_gap_list, patch_artist = True)
plt.ylim(0,200)
# plt.plot(np.linspace(0,1,10), mean_list)#, median_list)

Have a quick look at the distribution of ages:

In [ ]:
x_axis = np.arange(NUM_TASKS)+1
y_train_gt_axis = np.zeros(NUM_TASKS)
for i in range(NUM_TASKS):
    y_train_gt_axis[i] = np.sum(survival_times[:NUM_TRAIN] == x_axis[i])
plt.plot(x_axis, y_train_gt_axis)

In [ ]:
X_train = X[:NUM_TRAIN]
X_val = X[NUM_TRAIN:]
Y_train = Y[:NUM_TRAIN]
Y_val = Y[NUM_TRAIN:]

In [ ]:
np.set_printoptions(threshold=np.inf)
print(Kernel_Matrix_Train[0].shape)
print(Kernel_Matrix_Val[0])
age_gap = abs(survival_times[:NUM_TRAIN] - survival_times[0])
# print(survival_times - survival_times[0])
plt.scatter(Kernel_Matrix_Train[0], age_gap)


In [ ]:
def kernel_pegasos(batch_size, Kernel_Matrix, alpha, Y_truth, loss_list, iter_times, weight_decay):
    IDs = np.random.rand(batch_size,1) * NUM_TRAIN
    IDs = IDs.astype(int).reshape(-1)
    haty_IDs = (Kernel_Matrix[IDs].dot(alpha * Y_truth))  / (iter_times * weight_decay)
    mask = Y_truth[IDs]*haty_IDs < 1
    alpha[IDs] += mask
    
    '''

    alpha_copy = alpha.copy()
    alpha_copy[IDs] += mask
    
    W_product = (alpha_copy * Y_truth).T.dot(Kernel_Matrix.dot(alpha_copy * Y_truth))
    loss = 0.5 * np.sum(W_product.diagonal())
    hatY =  Kernel_Matrix.dot(alpha_copy * Y_truth) / ( iter_times * weight_decay)
    mask1 = Y_truth*hatY <  0
    loss += np.sum(abs((Y_truth*hatY))[mask1])
    
    if iter_times == 1 or loss < loss_list[-1]:
        loss_list.append(loss)
        # alpha = alpha_copy
        alpha[IDs] += mask
        # print("At iteration:",t, "the loss is,", loss)
        # print(alpha)
    '''

In [ ]:
alpha = np.zeros_like(Y_train) 
# alpha = np.ones_like(Y_train)
predict_age = np.zeros(NUM_EMPLOYEES) 
val_loss_list = []
index = 0
for t in range(1, max_iteration+1):
    # index = int(np.floor(np.random.rand()*NUM_TRAIN)) # randomly select one employee
    # index 
    kernel_pegasos(5, Kernel_Matrix_Train, alpha, Y_train, val_loss_list, t, weight_decay)
    # haty_index = (Kernel_Matrix_Train[index,:][np.newaxis,:].dot(alpha * Y_train)).reshape(-1) # calculate the predicted y vector
    # haty_index /= t * weight_decay

    ## update alpha

    # mask = Y_train[index,:] * haty_index < 1
    # alpha[index] += mask
    # index += 1
    # index = index%NUM_TRAIN
    
    if t%1000 == 0: # or t == 1:
        print("the iteration is:", t)
        hatY_train = Kernel_Matrix_Train.dot(alpha*Y_train) / ( t * weight_decay )
        
        hatY_val =  Kernel_Matrix_Val.dot(alpha*Y_train) / ( t * weight_decay )
        
        for i in range(NUM_TRAIN):
            predict_age[i] = np.where(hatY_train[i,:] < 0)[0][0] + 1

        for i in range(NUM_TRAIN,NUM_EMPLOYEES):
            predict_age[i] = np.where(hatY_val[i - NUM_TRAIN,:] < 0)[0][0] + 1
            
        acc_train = np.sum(Y_train * hatY_train > 0) / (NUM_TRAIN * NUM_TASKS)
        acc_val = np.sum(Y_val * hatY_val > 0) / (NUM_VAL * NUM_TASKS)
        print("the train accuracy is:", acc_train)
        print("the val accuracy is:", acc_val)
        
        useful_pair = 0.0
        denominator = NUM_VAL*(NUM_VAL-1)/2
        
        for i in range(NUM_TRAIN, NUM_EMPLOYEES):
            for j in range(i+1, NUM_EMPLOYEES):
                if (survival_times[i]-survival_times[j])*(predict_age[i]-predict_age[j])> 0: # useful pair
                    useful_pair += 1
                if  survival_times[i] == survival_times[j]: # two employees whose lifetime are identical
                    denominator -= 1
        
        c_index = useful_pair / denominator
        print("useful_pair", useful_pair)
        print("denominator", denominator)
        print("the c-index for VALIDATION is,",c_index)
        
  
print(np.sum(predict_age[NUM_TRAIN:] == survival_times[NUM_TRAIN:])/(NUM_VAL))

In [ ]:
print(np.sum(survival_times[NUM_TRAIN:] - predict_age[NUM_TRAIN:] < 0))
print(np.sum(survival_times[NUM_TRAIN:] - predict_age[NUM_TRAIN:] > 0))
print(print(np.sum(survival_times[NUM_TRAIN:] - predict_age[NUM_TRAIN:] == 0)))
print(survival_times[NUM_TRAIN:])
print(predict_age[NUM_TRAIN:])

In [ ]:
x_axis = np.arange(NUM_TASKS)+1 # the x axis
y_train_gt_axis = np.zeros(NUM_TASKS) # the ground truth train axis
y_val_gt_axis = np.zeros(NUM_TASKS) # the groud truth validation axis 
y_train_predict_axis = np.zeros(NUM_TASKS) # the predict train axis 
y_val_predict_axis = np.zeros(NUM_TASKS) # the predict validation axis 

for i in range(NUM_TASKS):
    y_train_gt_axis[i] = np.sum(survival_times[:NUM_TRAIN] == x_axis[i])
    y_val_gt_axis[i] = np.sum(survival_times[NUM_TRAIN:] == x_axis[i])
    y_train_predict_axis[i] = np.sum(predict_age[:NUM_TRAIN] == x_axis[i])
    y_val_predict_axis[i] = np.sum(predict_age[NUM_TRAIN:] == x_axis[i])
# plt.plot(x_axis, y_train_gt_axis, x_axis, y_train_predict_axis) # the training set: ground truth vs predict situation
# plt.plot(x_axis, y_val_gt_axis, x_axis, y_val_predict_axis) # the valiadtion set: ground truth vs predictsituation
plt.plot(x_axis, y_val_predict_axis)
# plt.plot(x_axis,y_train_predict_axis,y_val_predict_axis)